In [7]:
### Example of Miller with anthopods

from main import *

# Functions to compute endogenous variables
def compute_nb_legs(param):
    U = param['U']
    if U == "Spider":
        return 8
    elif U == "Beetle":
        return 6
    elif U == "Bee":
        return 6
    elif U == "Fly":
        return 6
    else:
        return None

def compute_stinger(param):
    U = param['U']
    if U == "Spider":
        return False
    elif U == "Beetle":
        return False
    elif U == "Bee":
        return True
    elif U == "Fly":
        return False
    else:
        return None

def compute_nb_eyes(param):
    U = param['U']
    if U == "Spider":
        return 8
    elif U == "Beetle":
        return 2
    elif U == "Bee":
        return 5
    elif U == "Fly":
        return 5
    else:
        return None

def compute_compound_eyes(param):
    U = param['U']
    if U == "Spider":
        return False
    elif U == "Beetle":
        return True
    elif U == "Bee":
        return True
    elif U == "Fly":
        return True
    else:
        return None

def compute_wings(param):
    U = param['U']
    if U == "Spider":
        return 0
    elif U == "Beetle":
        return 2
    elif U == "Bee":
        return 4
    elif U == "Fly":
        return 2
    else:
        return None

def compute_output(param):
    (nb_legs,stinger,nb_eyes,compound_eyes,wings) = (param["nb_legs"],param["stinger"],param["nb_eyes"],param["compound_eyes"],param["wings"],)
    if (nb_legs,stinger,nb_eyes,compound_eyes,wings) == (8,False,8,False,0):
        return "Spider"
    elif (nb_legs,stinger,nb_eyes,compound_eyes,wings) == (6,False,2,True,2):
        return "Beetle"
    elif (nb_legs,stinger,nb_eyes,compound_eyes,wings) == (6,True,5,True,4):
        return "Bee"
    elif (nb_legs,stinger,nb_eyes,compound_eyes,wings) == (6,False,5,True,2):
        return "Fly"
    else:
        return "Unknown"

# Exogenous variables
U = {'U': ["Spider", "Beetle", "Bee", "Fly"]}

# Endogenous variables
V = {'nb_legs' : [6, 8], \
    'stinger' : [True, False], \
    'nb_eyes' : [2, 5, 8], \
    'compound_eyes' : [True, False], \
    'wings' : [0, 2, 4], \
    'output': ["Spider", "Beetle", "Bee", "Fly","Unknown"]}

# Parent node for each endogenous variable
P = {'nb_legs' : (['U'],compute_nb_legs), \
    'stinger' : (['U'],compute_stinger), \
    'nb_eyes' : (['U'],compute_nb_eyes), \
    'compound_eyes' : (['U'],compute_compound_eyes), \
    'wings' : (['U'],compute_wings), \
    'output' : (['nb_legs','stinger','nb_eyes','compound_eyes','wings'],compute_output)}

C =  {'U' : ['nb_legs','stinger','nb_eyes','compound_eyes','wings'], \
    'nb_legs' : ['output'], \
    'stinger' : ['output'], \
    'nb_eyes' : ['output'], \
	'compound_eyes' : ['output'], \
    'wings' : ['output'],\
	'output' : []}

# Construction of situation (M,u)
Graph = CausalGraph(P,C)
Mod = Model(U,V,Graph)

#First Situation
u = {'U': "Spider"}
v = {'nb_legs' : 8, \
    'stinger' : False, \
    'nb_eyes' : 8, \
    'compound_eyes' : False, \
    'wings' : 0, \
    'output': "Spider"}

Sit = Situation(Mod,u,v)

#Second Situation
u1 = {'U': "Bee"}
v1 = {'nb_legs' : 6, \
    'stinger' : True, \
    'nb_eyes' : 5, \
    'compound_eyes' : True, \
    'wings' : 4, \
    'output': "Bee"}

Sit1 = Situation(Mod,u1,v1)

## Fonction de calculs de base

### Calcul de valeurs d'une variable

In [8]:
print(Sit.v)
Sit.reset_val_v()
print(Sit.v)
print(value('nb_legs',Sit)) #a cet instant on ne connait aucune valeurs des variables de V
                            #on calcule les valeurs des variables a partir des variables exogênes.
print(value('output',Sit)) #de même pour l'output

Sit.set_val_v()
print(Sit.v)

{'nb_legs': 8, 'stinger': False, 'nb_eyes': 8, 'compound_eyes': False, 'wings': 0, 'output': 'Spider'}
{'nb_legs': None, 'stinger': None, 'nb_eyes': None, 'compound_eyes': None, 'wings': None, 'output': None}
8
Spider
{'nb_legs': 8, 'stinger': False, 'nb_eyes': 8, 'compound_eyes': False, 'wings': 0, 'output': 'Spider'}


### Verification d'un fait

In [10]:
fact = {'output': "Bee"}
foil = {'output': "Fly"}
print(check(fact,Sit1))
print(check(foil,Sit1))
print(check_not(fact,Sit1))
print(check_not(foil,Sit1))

True
False
False
True


## Générateurs de causes

### Causes effectives (AC)

In [12]:
print(actual_cause_generator_v2(fact,Sit1)) #on utilise la v2 car la première version de AC2 ne semble pas fonctionner.

[{'wings': 4}, {'compound_eyes': True}, {'nb_eyes': 5}, {'stinger': True}, {'nb_legs': 6}]


On peut voir qu'il y a 5 causes effectives du fait ```{'output': "Bee"}```.<br>
Dans cet exemple on ne peut pas regarder des causes d'autres variables endogènes car le graphe n'est pas assez profond.

### Causes contrefactuelles (CC)

In [15]:
print(counterfactual_cause_generator(fact,foil,Sit1))

[({'wings': 4}, {'wings': 2}), ({'stinger': True}, {'stinger': False})]


On retrouve bien les 2 même causes données par Miller dans l'exemple.